<a href="https://colab.research.google.com/github/viki6666/Pytorch_learn/blob/master/CNN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import torch
from torch import nn, optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets

batch_size = 128
learning_rate = 0.01
num_epoch = 10

# 实例化MNIST数据集对象
train_data = datasets.MNIST('./dataset', train=True, transform=transforms.ToTensor(), download=True)
test_data = datasets.MNIST('./dataset', train=False, transform=transforms.ToTensor(), download=True)

# train_loader：以batch_size大小的样本组为单位的可迭代对象
train_loader = DataLoader(train_data, batch_size, shuffle=True)
test_loader = DataLoader(test_data)

class CNN(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_dim, 6, 3, stride=1, padding=1)
        self.batch_norm1 = nn.BatchNorm2d(6)
        self.relu = nn.ReLU(True)
        self.conv2 = nn.Conv2d(6, 16, 5, stride=1, padding=0)
        self.pool = nn.MaxPool2d(2, 2)
        self.batch_norm2 = nn.BatchNorm2d(16)

        self.fc1 = nn.Linear(400, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, out_dim)

    def forward(self, x):
        x = self.batch_norm1(self.conv1(x))
        x = F.relu(x)
        x = self.pool(x)
        x = self.batch_norm2(self.conv2(x))
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def print_model_name(self):
        print("Model Name: CNN")


class Cnn(nn.Module):
    def __init__(self, in_dim, n_class):
        super(Cnn, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(in_dim, 6, 3, stride=1, padding=1),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2),
            nn.Conv2d(6, 16, 5, stride=1, padding=0),
            nn.ReLU(True),
            nn.MaxPool2d(2, 2))

        self.fc = nn.Sequential(
            nn.Linear(400, 120), nn.Linear(120, 84), nn.Linear(84, n_class))

    def forward(self, x):
        # print(x.size()) torch.Size([1024, 1, 28, 28])
        out = self.conv(x)
        out = out.view(out.size(0), -1)
        # print(out.size()) = torch.Size([1024, 400])
        out = self.fc(out)
        # print(out.size()) torch.Size([1024, 10])
        return out

    def print_model_name(self):
        print("Model Name: Cnn")


isGPU = torch.cuda.is_available()
print(isGPU)
model = CNN(1, 10)
if isGPU:
    model = model.cuda()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=learning_rate)
for epoch in range(num_epoch):
    running_acc = 0.0
    running_loss = 0.0
    for i, data in enumerate(train_loader, 1): # train_loader：以batch_size大小的样本组为单位的可迭代对象
        img, label = data
        img = Variable(img)
        label = Variable(label)
        if isGPU:
            img = img.cuda()
            label = label.cuda()
        # forward
        out = model(img)
        loss = criterion(out, label)
        # print(label)
        # backward
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
	
        _, pred = torch.max(out, dim=1)  # 按维度dim 返回最大值
        running_loss += loss.item()*label.size(0)
        current_num = (pred == label).sum() # variable
        acc = (pred == label).float().mean()        # variable
        running_acc += current_num.item()

        if i % 100 == 0:
            print("epoch: {}/{}, loss: {:.6f}, running_acc: {:.6f}"
                  .format(epoch+1, num_epoch, loss.item(), acc.item()))
    print("epoch: {}, loss: {:.6f}, accuracy: {:.6f}".format(epoch+1, running_loss, running_acc/len(train_data)))

model.eval()
current_num = 0
for i , data in enumerate(test_loader, 1):
    img, label = data
    if isGPU:
        img = img.cuda()
        label = label.cuda()
    with torch.no_grad():
        img = Variable(img)
        label = Variable(label)
    out = model(img)
    _, pred = torch.max(out, 1)
    current_num += (pred == label).sum().item()

print("Test result: accuracy: {:.6f}".format(float(current_num/len(test_data))))

torch.save(model.state_dict(), './cnn.pth') # 保存模型

  0%|          | 16384/9912422 [00:00<01:11, 138071.04it/s]

9920512it [00:00, 31112874.80it/s]                          


Extracting ./dataset/MNIST/raw/train-images-idx3-ubyte.gz


32768it [00:00, 425638.05it/s]
0it [00:00, ?it/s]

Extracting ./dataset/MNIST/raw/train-labels-idx1-ubyte.gz


1654784it [00:00, 7120100.81it/s]                            
8192it [00:00, 178530.17it/s]


Extracting ./dataset/MNIST/raw/t10k-images-idx3-ubyte.gz
Extracting ./dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz
Processing...
Done!
False
epoch: 1/10, loss: 2.234754, running_acc: 0.250000
epoch: 1/10, loss: 2.095104, running_acc: 0.539062
epoch: 1/10, loss: 1.756960, running_acc: 0.648438
epoch: 1/10, loss: 1.154220, running_acc: 0.742188
epoch: 1, loss: 109239.954239, accuracy: 0.514450
epoch: 2/10, loss: 0.524628, running_acc: 0.914062
epoch: 2/10, loss: 0.561948, running_acc: 0.859375
epoch: 2/10, loss: 0.264466, running_acc: 0.937500
epoch: 2/10, loss: 0.340932, running_acc: 0.906250
epoch: 2, loss: 25983.313155, accuracy: 0.893967
epoch: 3/10, loss: 0.246970, running_acc: 0.937500
epoch: 3/10, loss: 0.170294, running_acc: 0.945312
epoch: 3/10, loss: 0.169994, running_acc: 0.945312
epoch: 3/10, loss: 0.154523, running_acc: 0.945312
epoch: 3, loss: 11597.064387, accuracy: 0.946917
epoch: 4/10, loss: 0.149828, running_acc: 0.937500
epoch: 4/10, loss: 0.092835, running_acc: 0.97656

In [0]:
import torch
from torch import nn, optim # 网络及优化器
import torch.nn.functional as F # 网络中不需要更新参数的函数
from torch.autograd import Variable
from torchvision import datasets # 处理数据集
from torch.utils.data import DataLoader # 加载数据集
from torchvision import transforms 

batch_size = 128
learning_rate = 0.01 # 梯度下降的学习率
num_epoch = 10

#数据实例化
train_data =datasets.MNIST('./dataset',train=True,transform=transforms.ToTensor(),download=True)
test_data=datasets.MNIST('./dataset',train=False,transform=transforms.ToTensor(),download=True)

#数据加载
train_loader=DataLoader(train_data,batch_size,shuffle=True)
test_loader=DataLoader(test_data)

class CNN(nn.Module):
  def __int__(self,in_dim,out_dim):
    super(CNN,self).__init__()
    self.conv1=nn.Conv2d(in_dim,6,3,stride=1,padding=1)
    self.batch_norm1=nn.BatchNorm2d(6)#归一化
    self.relu=nn.ReLU(True)#激活函数
    self.conv2 = nn.Conv2d(6, 16, 5, stride=1, padding=0)
    self.pool=nn.Maxpool2d(2,2)
    self.batch_norm2=nn.BatchNorm2d(16)

    self.fc1=nn.Linear(400,120)
    self.fc2=nn.Linear(120，84)
    
    



In [0]:
class CNN(nn.Module):
    def __init__(self, in_dim, out_dim):
        super(CNN, self).__init__()
        self.conv1 = nn.Conv2d(in_dim, 6, 3, stride=1, padding=1)
        self.batch_norm1 = nn.BatchNorm2d(6)
        self.relu = nn.ReLU(True)
        self.conv2 = nn.Conv2d(6, 16, 5, stride=1, padding=0)
        self.pool = nn.MaxPool2d(2, 2)
        self.batch_norm2 = nn.BatchNorm2d(16)

        self.fc1 = nn.Linear(400, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, out_dim)

    def forward(self, x):
        x = self.batch_norm1(self.conv1(x))
        x = F.relu(x)
        x = self.pool(x)
        x = self.batch_norm2(self.conv2(x))
        x = self.relu(x)
        x = self.pool(x)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def print_model_name(self):
        print("Model Name: CNN")

In [0]:
tensor
nn.ReLU